# Extract subset of ozone profiles from UKESM1

This notebook illustrates how to extract a subset of ozone profiles from UKESM1. It is designed to be implemented on a Pangeo service. Note that it also uses a "preprocessing" script from Julian Busecke. More up-to-date versions can be found here: https://github.com/jbusecke/cmip6_preprocessing

This notebook extracts the UKESM1 profiles from three different experiments, calculates the annual mean profiles, and saves the results to NetCDF files. These NetCDF files are then used by notebook 2.0 to train the unsupervised classification analysis. 

## Import required packages

In [1]:
import intake
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
import xgcm
import dask
import pprint
import gsw
import xesmf as xe
from xhistogram.xarray import histogram
from datetime import datetime

# local file for CMIP6 preprocessing
from cmip6_preprocessing.preprocessing import combined_preprocessing

## Locate UKESM1 ozone data

We select the MOHC (Met Office - Hadley Centre) created model UKESM1. We select the monthly atmospheric variables (just O3) from three different experiments. Specifically, we select the historical run, SSP126, and SSP585. 

In [2]:
col_url='https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json'
col = intake.open_esm_datastore(col_url)
cat = col.search(institution_id='MOHC',
                 source_id='UKESM1-0-LL',
                 table_id='Amon',
                 experiment_id=['historical','ssp126','ssp585'],
                 variable_id=['o3'],
                 member_id=['r1i1p1f2'],
                 grid_label='gn')
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,MOHC,UKESM1-0-LL,historical,r1i1p1f2,Amon,o3,gn,gs://cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/histori...,NaN,20190406
1,ScenarioMIP,MOHC,UKESM1-0-LL,ssp126,r1i1p1f2,Amon,o3,gn,gs://cmip6/CMIP6/ScenarioMIP/MOHC/UKESM1-0-LL/...,NaN,20190503
2,ScenarioMIP,MOHC,UKESM1-0-LL,ssp585,r1i1p1f2,Amon,o3,gn,gs://cmip6/CMIP6/ScenarioMIP/MOHC/UKESM1-0-LL/...,NaN,20190507


In [3]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': True},
                                cdf_kwargs={'chunks': {}, 'decode_times': True})
dset_dict.keys()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


dict_keys(['CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn', 'ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Amon.gn', 'ScenarioMIP.MOHC.UKESM1-0-LL.ssp126.Amon.gn'])

## Preprocess the historical, ssp126, and ssp585 datasets

In [4]:
historical = dset_dict['CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn']
ssp126 = dset_dict['ScenarioMIP.MOHC.UKESM1-0-LL.ssp126.Amon.gn']
ssp585 = dset_dict['ScenarioMIP.MOHC.UKESM1-0-LL.ssp585.Amon.gn']

historical = combined_preprocessing(historical)
ssp126 = combined_preprocessing(ssp126)
ssp585 = combined_preprocessing(ssp585)

## Drop the coordinates that are not needed

In [5]:
historical = historical.drop(('lon_bounds','time_bounds','lat_bounds','member_id','bnds','lon_verticies','lat_verticies','vertex'))
ssp126 = ssp126.drop(('lon_bounds','time_bounds','lat_bounds','member_id','bnds','lon_verticies','lat_verticies','vertex'))
ssp585 = ssp585.drop(('lon_bounds','time_bounds','lat_bounds','member_id','bnds','lon_verticies','lat_verticies','vertex'))

## Calculate annual mean profiles

In [6]:
historical = historical.groupby('time.year').mean('time')
ssp126 = ssp126.groupby('time.year').mean('time')
ssp585 = ssp585.groupby('time.year').mean('time')

## Calculate decadal means

In [7]:
historical = historical.sel(year=slice(2004,2014)).mean(dim='year',skipna=True)
ssp126 = ssp126.sel(year=slice(2090,2100)).mean(dim='year',skipna=True)
ssp585 = ssp585.sel(year=slice(2090,2100)).mean(dim='year',skipna=True)

In [8]:
historical = historical.squeeze()
ssp126 = ssp126.squeeze()
ssp585 = ssp585.squeeze()

In [9]:
# scale to get units in ppmv ()
#historical['o3_ppm'] = historical.o3*1e6
#ssp126['o3_ppm'] = ssp126.o3*1e6
#ssp585['o3_pppm'] = ssp585.o3*1e6

# scale to get units in mPa
historical['o3'] = historical.o3*historical.plev/1e-3
ssp126['o3'] = ssp126.o3*ssp126.plev/1e-3
ssp585['o3'] = ssp585.o3*ssp585.plev/1e-3

# variable to indicate which experiment the data is from
historical['experiment'] = 'hist'
ssp126['experiment'] = 'ssp126'
ssp585['experiment'] = 'ssp585'

## Prepare attributes for combined NetCDF file

In [10]:
historical.attrs['Prepared by'] = 'D. Jones'
historical.attrs['Institute'] = 'British Antarctic Survey'
historical.o3.attrs['Units'] = 'hPa'
historical.o3.attrs['Long name'] = 'Partial pressure of ozone'
historical.attrs['Model Info'] = 'UK Earth System Model 1'
historical.attrs['Description'] = 'Decadal mean ozone profiles from historical experiment'
historical.attrs['Years covered'] = '2004-2014'

ssp126.attrs['Prepared by'] = 'D. Jones'
ssp126.attrs['Institute'] = 'British Antarctic Survey'
ssp126.o3.attrs['Units'] = 'hPa'
ssp126.o3.attrs['Long name'] = 'Partial pressure of ozone'
ssp126.attrs['Model Info'] = 'UK Earth System Model 1'
ssp126.attrs['Description'] = 'Decadal mean ozone profiles from ssp126 experiment (strong emissions reductions)'
ssp126.attrs['Years covered'] = '2090-2100'

ssp585.attrs['Prepared by'] = 'D. Jones'
ssp585.attrs['Institute'] = 'British Antarctic Survey'
ssp585.o3.attrs['Units'] = 'hPa'
ssp585.o3.attrs['Long name'] = 'Partial pressure of ozone'
ssp585.attrs['Model Info'] = 'UK Earth System Model 1'
ssp585.attrs['Description'] = 'Decadal mean ozone profiles from ssp585 experiment (strong emissions reductions)'
ssp585.attrs['Years covered'] = '2090-2100'


## Save results to NetCDF files

In [11]:
historical.load()
#historical.to_netcdf(path='UKESM_O3_historical.nc')
ssp126.load()
#ssp126.to_netcdf(path='UKESM_O3_ssp126.nc')
ssp585.load()
#ssp585.to_netcdf(path='UKESM_O3_ssp585.nc')

<xarray.Dataset>
Dimensions:     (y: 144, x: 192, plev: 19)
Coordinates:
  * y           (y) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * x           (x) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1
  * plev        (plev) float64 1e+05 9.25e+04 8.5e+04 ... 1e+03 500.0 100.0
    lon         (x, y) float64 0.9375 0.9375 0.9375 0.9375 ... 359.1 359.1 359.1
    lat         (x, y) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
Data variables:
    o3          (plev, y, x) float64 nan nan nan nan ... 0.3113 0.3112 0.3112
    experiment  <U6 'ssp585'
Attributes:
    Prepared by:    D. Jones
    Institute:      British Antarctic Survey
    Model Info:     UK Earth System Model 1
    Description:    Annual mean ozone profiles from ssp585 experiment (strong...
    Years covered:  2090-2100

## Create merged dataset, save to single NetCDF file

In [12]:
merged = xr.concat([historical,ssp126,ssp585],dim='experiment')
merged.to_netcdf(path='UKESM_O3_merged.nc')
merged

<xarray.Dataset>
Dimensions:     (y: 144, x: 192, experiment: 3, plev: 19)
Coordinates:
  * y           (y) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * x           (x) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1
  * plev        (plev) float64 1e+05 9.25e+04 8.5e+04 ... 1e+03 500.0 100.0
    lon         (x, y) float64 0.9375 0.9375 0.9375 0.9375 ... 359.1 359.1 359.1
    lat         (x, y) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * experiment  (experiment) <U6 'hist' 'ssp126' 'ssp585'
Data variables:
    o3          (experiment, plev, y, x) float64 nan nan nan ... 0.3112 0.3112
Attributes:
    Prepared by:    D. Jones
    Institute:      British Antarctic Survey
    Model Info:     UK Earth System Model 1
    Description:    Annual mean ozone profiles from historical experiment
    Years covered:  2004-2014